# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [12]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow import random
import os, logging, glob
from _data import organise_images_in_folders
from _logging import set_logging

set_logging(logging)
set_up_flag = True
if set_up_flag:
    organise_images_in_folders()

Folder exists for training images, no need to organise data


### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [13]:
#set a seed
seed = 42
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [14]:
rowsOfImage, columnsOfImage = 64, 64
input_shape = (rowsOfImage, columnsOfImage, 3)
classifier.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape))

### Step 4: Now, add the pooling layer with the image size as 2x2

In [15]:
classifier.add(MaxPool2D(pool_size = (2, 2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [16]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [17]:
classifier.add(Dense(128, activation = 'relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [18]:
classifier.add(Dense(1, activation="sigmoid"))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [19]:
optimizerAdam = Adam(name = 'accuracy')

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [20]:
path = "car_flower_small"
dir_list = os.listdir(path)
i = 0

In [21]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

In [22]:
# this is a generator that will read pictures found in
# subfolders of 'data/train', and indefinitely generate
# batches of augmented image data
batch_size = 32
image_size = (64, 64)
train_directory = os.path.join("car_flower_small", "data", "trainset")
train_generator = train_datagen.flow_from_directory(
        train_directory,  # this is the target directory
        target_size=image_size,  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
i = 0


Found 1498 images belonging to 2 classes.


### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

In [23]:
# this is a similar generator, for validation data
test_directory = os.path.join("car_flower_small", "data", "testset")
test_generator = test_datagen.flow_from_directory(
        test_directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary')

Found 500 images belonging to 2 classes.


### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.

In [25]:
classifier.compile(loss='binary_crossentropy',
              optimizer=optimizerAdam,
              metrics=['accuracy'])

_train_steps_per_epoch = len(train_generator.filenames) // batch_size
_test_steps_per_epoch = len(test_generator.filenames) // batch_size
logging.info(f"train_steps_per_epoch = {_train_steps_per_epoch}")
logging.info(f"test_steps_per_epoch = {_test_steps_per_epoch}")
classifier.fit_generator(
        train_generator,
        steps_per_epoch=_train_steps_per_epoch,
        epochs=50,
        validation_data=test_generator,
        validation_steps=_test_steps_per_epoch)

2023-01-17 20:52:48,241 | INFO : train_steps_per_epoch = 46
2023-01-17 20:52:48,245 | INFO : test_steps_per_epoch = 15
Epoch 1/50


C:\Users\hijik\AppData\Local\Temp\ipykernel_22652\2125813100.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


20/46 [============>.................] - ETA: 2s - loss: 0.6103 - accuracy: 0.6735

UnknownError: Graph execution error:

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000156DD912540>
Traceback (most recent call last):

  File "c:\Users\hijik\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\Users\hijik\Anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\hijik\Anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\hijik\Anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 901, in wrapped_generator
    for data in generator_fn():

  File "c:\Users\hijik\Anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 1048, in generator_fn
    yield x[i]

  File "c:\Users\hijik\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 116